In [1]:
def training(trainX,trainZ,k,startprob_org,transmat_org,means_org,covars_org):
    print "fitting these numbers to HMM"
    model_p = GaussianHMM(n_components=k, covariance_type="diag", n_iter=1000).fit(trainX)
    #evaluating training
    hidden_states = model_p.predict(trainX)
    transmat=model_p.transmat_
    means=model_p.means_
    covars=model_p.covars_
    startprob=model_p.startprob_

    l = list(permutations(range(0, k)))
    min_means=1000
    min_covars=0

    for p in l:

        var_means=means
        var_means[:,:]=var_means[p,:]
        print "means ",norm(var_means-means_org,2)
        if(min_means>norm(var_means-means_org,2)):
            min_means=norm(var_means-means_org,2)
            print "min means changed",min_means
            perm_m=p

    var_start=startprob.reshape(k,1)
    var_start[:,:]=var_start[perm_m,:]
    min_start=norm(var_start-startprob_org.reshape(k,1),1)

    var_trans=transmat
    var_trans[:,:]=var_trans[perm_m,:]
    var_trans[:,:]=var_trans[:,perm_m]
    min_transmat=norm(var_trans-transmat_org,1)

    var_covars=covars
    var_covars[:,:,:]=var_covars[p,:,:]
    for i in range(k):
        min_covars=min_covars+norm(var_covars[i]-covars_org[i],2)
    min_covars=min_covars/k

    var_trainZ=trainZ
    for i in range(trainZ.shape[0]):
        var_trainZ[i]=perm_m[trainZ[i]]
    acctrain=norm(hidden_states-var_trainZ,1)

    return min_start,min_transmat,min_means,min_covars,acctrain,model_p,perm_m


In [5]:
import numpy as np
from hmmlearn.hmm import GaussianHMM
from itertools import permutations
from scipy.linalg import norm
from hmmlearn import hmm
import math
from datetime import datetime
k=4
n=500
l=2
trainX, trainZ,startprob_org,transmat_org,means_org,covars_org,testX, testZ=createSample(n,l,k)
min_start, min_transmat, min_means, min_covars, acctrain, model_p, perm_m = training(trainX, trainZ, k,
                                                                                         startprob_org, transmat_org,
                                                                                         means_org, covars_org)

fitting these numbers to HMM
means  16.5334265987
min means changed 16.5334265987
means  19.0940949228
means  16.7521557774
means  16.5334265987
means  16.7521557774
means  20.7597203408
means  20.3582650505
means  23.2140019066
means  19.9079944228
means  12.9348660942
min means changed 12.9348660942
means  23.2140019066
means  16.5391661667
means  20.0118796423
means  15.4887474173
means  16.7521557774
means  22.0228776225
means  20.3582650505
means  15.4887474173
means  16.5391661667
means  23.2140019066
means  20.0052114569
means  23.0429203448
means  16.6759316006
means  19.9079944228


/Users/Madhu/Downloads/softwares/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:32: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Madhu/Downloads/softwares/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:33: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Madhu/Downloads/softwares/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:34: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/Madhu/Downloads/softwares/anaconda3/envs/python2/lib/python2.7/site-packages/ipykernel/__main__.py:35: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [3]:
def createSample(n,l,k):
    startprob_org = np.array([0.6, 0.3, 0.1, 0.0])
    # The transition matrix, note that there are no transitions possible
    # between component 1 and 3
    transmat_org = np.array([[0.7, 0.2, 0.0, 0.1],
                         [0.3, 0.5, 0.2, 0.0],
                         [0.0, 0.3, 0.5, 0.2],
                         [0.2, 0.0, 0.2, 0.6]])
    # The means of each component
    means_org = np.array([[0.0,  0.0],
                      [0.0, 11.0],
                      [9.0, 10.0],
                      [11.0, -1.0]])
    # The covariance of each component
    covars_org = .5 * np.tile(np.identity(2), (4, 1, 1))

    # Build an HMM instance and set parameters
    model = hmm.GaussianHMM(n_components=4, covariance_type="full")
    model.startprob_ = startprob_org
    model.transmat_ = transmat_org
    model.means_ = means_org
    model.covars_ = covars_org

    # Generate samples
    X, Z = model.sample(500)

    # compute how much of the data is training and testing
    train_rows = math.floor(0.6 * X.shape[0])
    test_rows = X.shape[0] - train_rows

    # separate out training and testing data
    trainX = X[:train_rows]
    trainZ = Z[:train_rows]
    testX = X[train_rows:]
    testZ = Z[train_rows:]
    return trainX,trainZ,startprob_org,transmat_org,means_org,covars_org,testX,testZ

In [6]:
print model_p

GaussianHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
      covars_weight=1, init_params='stmc', means_prior=0, means_weight=0,
      min_covar=0.001, n_components=4, n_iter=1000, params='stmc',
      random_state=None, startprob_prior=1.0, tol=0.01, transmat_prior=1.0,
      verbose=False)


print model_p.type

In [8]:
print type(model_p)

<class 'hmmlearn.hmm.GaussianHMM'>


In [9]:
def testing(testX, testZ, k, model_p,perm_m):
    print "fitting these numbers to HMM"
    #model_p = GaussianHMM(n_components=k, covariance_type="diag", n_iter=1000).fit(trainX)
    # evaluating training
    predZ = model_p.predict(testX)
    var_testZ=testZ
    for i in range(testZ.shape[0]):
        var_testZ[i]=perm_m[testZ[i]]
    acctest=norm(predZ-var_testZ,1)

    return acctest
